In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
import multiprocessing as mp
import time
import glob
import re
import random
from sklearn.model_selection import train_test_split
import torch

sys.path.insert(0, '/Data/user/panhailin/git_lab/npspy')
import npspy as nps

# 全局配置

In [2]:
torch.set_num_threads(10)

all_peps = [
    'hp1_1', 'hp1_2', 'hp1_3', 'hp1_4', 'hp1_5', 'hp1_6', 'hp1_7', 'hp1_8', 'hp1_9',
    'hp2_1', 'hp2_2', 'hp2_3', 'hp2_4', 'hp2_5', 'hp2_6',
]

y_code_dict = nps.ml.set_y_codes_for_classes(np.array(all_peps)[:,None])
y_code_dict
y_to_label_dict = {v:k for k,v in y_code_dict.items()}
y_code_dict, y_to_label_dict

({np.str_('hp1_1'): 0,
  np.str_('hp1_2'): 1,
  np.str_('hp1_3'): 2,
  np.str_('hp1_4'): 3,
  np.str_('hp1_5'): 4,
  np.str_('hp1_6'): 5,
  np.str_('hp1_7'): 6,
  np.str_('hp1_8'): 7,
  np.str_('hp1_9'): 8,
  np.str_('hp2_1'): 9,
  np.str_('hp2_2'): 10,
  np.str_('hp2_3'): 11,
  np.str_('hp2_4'): 12,
  np.str_('hp2_5'): 13,
  np.str_('hp2_6'): 14},
 {0: np.str_('hp1_1'),
  1: np.str_('hp1_2'),
  2: np.str_('hp1_3'),
  3: np.str_('hp1_4'),
  4: np.str_('hp1_5'),
  5: np.str_('hp1_6'),
  6: np.str_('hp1_7'),
  7: np.str_('hp1_8'),
  8: np.str_('hp1_9'),
  9: np.str_('hp2_1'),
  10: np.str_('hp2_2'),
  11: np.str_('hp2_3'),
  12: np.str_('hp2_4'),
  13: np.str_('hp2_5'),
  14: np.str_('hp2_6')})

In [3]:
train_objs = [f"../../../03.results/classification_on_clean_data/hp12/{pep}/{pep}_valid80_clean_obj.pkl" for pep in all_peps]
test_objs = [f"../../../03.results/classification_on_clean_data/hp12/{pep}/{pep}_valid20_clean_obj.pkl" for pep in all_peps]
labels = all_peps

In [4]:
def stratified_sample(df, column_name, sample_size=15000, random_state=42):
    """
    对DataFrame按指定列类别分层随机抽样
    
    参数:
        df: 输入DataFrame
        column_name: 分层依据的列名
        sample_size: 每类抽取样本数(默认15000)
        random_state: 随机种子
    
    返回:
        抽样后的新DataFrame
    """
    re_df = df.groupby(column_name, group_keys=True).apply(
        lambda x: x.sample(min(len(x), sample_size), 
                          random_state=random_state),
        include_groups=False,
    )
    re_df[re_df.index.names[0]] =  [i[0] for i in re_df.index]
    re_df.index = [i[1] for i in re_df.index]
    return re_df

In [21]:
def train_pipeline(train_objs, test_objs, labels, y_code_dict, all_peps, train_name='clean_data', train_sample_size=14000, seed=42):
    # 读取pkl文件，生成readid，X，y组成的df
    train_df = nps.ml.get_X_y_from_objs(objs=train_objs, labels=labels, y_code_dict=y_code_dict, down_sample_to=1000, att='signal')
    train_df = stratified_sample(train_df, 'y', sample_size=train_sample_size, random_state=seed)
    train_df, valid_df = train_test_split(train_df, test_size=1/8, random_state=seed, stratify=train_df['y'])
    test_df = nps.ml.get_X_y_from_objs(objs=test_objs, labels=labels, y_code_dict=y_code_dict, down_sample_to=1000, att='signal')
    test_df = stratified_sample(test_df, 'y', sample_size=6000, random_state=seed)

    # 通过data_df构建dataloader
    batch_size = 128
    train_dl = nps.ml.construct_dataloader_from_data_df(train_df, batch_size=batch_size, augment=False)
    valid_dl = nps.ml.construct_dataloader_from_data_df(valid_df, batch_size=batch_size)
    test_dl = nps.ml.construct_dataloader_from_data_df(test_df, batch_size=batch_size, shuffle=False)

    # train
    nps.ml.seed_everything(seed)
    clf = nps.ml.Trainer(lr=0.005, num_classes=len(all_peps), epochs=200, device='cuda', lr_scheduler_patience=3, label_smoothing=0.1, model_name='CNN1DL1000')
    clf.fit(train_dl, valid_dl, early_stopping_patience=30, name=train_name)

    # pred
    pred_df = clf.predict(test_dl, name=train_name, y_to_label_dict=y_to_label_dict)
    test_all_reads_s = pred_df['true'].value_counts()
    cm_df = nps.ml.get_cm(pred_df, label_order=all_peps)
    cm_df.to_csv(f"../../../03.results/classification_on_clean_data/hp12/diff_data_size/clean/{train_name}_cm.csv")
    acc = np.sum(np.diag(cm_df))/len(pred_df)
    print(f'{train_sample_size}: {acc}')
    pred_proba_df = clf.predict_proba(test_dl, name=train_name)
    pred_proba_df.to_csv(f"../../../03.results/classification_on_clean_data/hp12/diff_data_size/clean/{train_name}_pred_proba.csv")
    return acc


In [22]:
acc_df = []
# train_sample_sizes = [14000, 12000, 10000, 8000, 6000, 4000]
# train_sample_sizes = [4000, 6000, 8000, 10000, 12000, 14000]
train_sample_sizes = [25000, 20000, 15000, 10000, 5000]

for train_sample_size in train_sample_sizes:
    acc = train_pipeline(train_objs, test_objs, labels, y_code_dict, all_peps, train_name=f'clean_data_ds_{train_sample_size}', train_sample_size=train_sample_size)
    acc_df.append([train_sample_size, acc])
acc_df =pd.DataFrame(acc_df)
acc_df.columns = ['train_sample_size', 'acc']
acc_df.to_csv("../../../03.results/classification_on_clean_data/hp12/diff_data_size/clean/acc.csv")
acc_df

Model CNN1DL1000 has total parameter number: 6.21 M
Epoch   0 / 200 train_loss: 1.1765 train_acc: 0.7502 val_loss: 1.0903 val_acc: 0.7832 lr: 0.005
Epoch   1 / 200 train_loss: 0.8916 train_acc: 0.8720 val_loss: 1.1603 val_acc: 0.7499 lr: 0.005
Epoch   2 / 200 train_loss: 0.8187 train_acc: 0.9010 val_loss: 1.5012 val_acc: 0.6511 lr: 0.005
Epoch   3 / 200 train_loss: 0.7794 train_acc: 0.9160 val_loss: 0.7806 val_acc: 0.9125 lr: 0.005
Epoch   4 / 200 train_loss: 0.7528 train_acc: 0.9261 val_loss: 2.1566 val_acc: 0.4659 lr: 0.005
Epoch   5 / 200 train_loss: 0.7315 train_acc: 0.9344 val_loss: 0.9016 val_acc: 0.8544 lr: 0.005
Epoch   6 / 200 train_loss: 0.7159 train_acc: 0.9405 val_loss: 1.1812 val_acc: 0.7516 lr: 0.005
Epoch   7 / 200 train_loss: 0.7029 train_acc: 0.9456 val_loss: 0.7719 val_acc: 0.9143 lr: 0.005
Epoch   8 / 200 train_loss: 0.6910 train_acc: 0.9505 val_loss: 1.6486 val_acc: 0.5971 lr: 0.005
Epoch   9 / 200 train_loss: 0.6815 train_acc: 0.9546 val_loss: 0.7536 val_acc: 0.922

,train_sample_size,acc
0,25000,0.952322
1,20000,0.940056
2,15000,0.938778
3,10000,0.935567
4,5000,0.912622
